# Data Generator

In [10]:
import os
import json
import sys
from pprint import pprint # Delete

In [11]:
with open("config.json", "r") as read_file:
    conf = json.load(read_file)
#    pprint(conf)

data = conf['data']
port = conf['port']
#port


In [12]:
### Profile template

## General

def prefix():
    h = '@prefix : <#>.\n\
@prefix foaf: <http://xmlns.com/foaf/0.1/>.\n\
@prefix dct:  <http://purl.org/dc/terms/>.\n\
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.\n\
@prefix sioc: <http://rdfs.org/sioc/ns#>.\n\
@prefix sioct: <http://rdfs.org/sioc/types#>.\n\
\n'
    return h

def body(c):
    return ':me a foaf:Person;\n\
    foaf:name \'Person' + str(c) + '\';\n'

## Knowing
def knows(pods):
    k = ''
    for p in range(pods):
        #creating a List of users to know
        k += '    foaf:knows <http://localhost:' + str(port) + '/person' + str(p) + '/profile.ttl#me>;\n'
    return k


def account(c):
    return '    sioc:UserAccount :user' + str(c) + '.\n\n'
    
    
def user(c, n_posts):
    u = ':user' + str(c) + '\n'
    for p in range(n_posts):
        u += '    sioc:creator_of :post' + str(p) + ';\n'
    u += '    sioct:Microblog :MbPrivate;\n'
    u += '    sioct:Microblog :MbOfficial.\n'
    return u

## Posts
def body_post(c):
    return ':post' + str(c) + '\n\
    a sioct:MicroblogPost;\n\
    sioc:content "Lorem Ipsum";\n\
    dct:created "2020-01-01";\n\
    sioc:topic <http://example.org/topics/hcls>.\n'  ### CHANGE!
    # sioc:has_container <http://localhost:8080/person' + str(c) + '/profile/microblog>. #hosting the server at which level?
    


In [13]:
## ACL File

def prefix_acl():
    return '@prefix : <#>.\n\
@prefix acl: <http://www.w3.org/ns/auth/acl#>.\n\
@prefix ppo: <http://vocab.deri.ie/ppo#>.\n\
\n'

def body_acl(c, auths, pods):

    b = ':acl1 a acl:Authorization;\n\
    acl:mode acl:Read;\n'
    
    for p in range(pods):
        b += '    acl:agent <http://localhost:' + str(port) + '/person' + str(p) + '/profile.ttl#me>;\n'
    
    b += '    ppo:appliesToNamedGraph <http://localhost:' + str(port) + '/person' + str(c) + '/profile.ttl#Card>'
     
    if auths == 1:
        b += '.\n'
    if auths == 2:
        b += ';\n\
    ppo:appliesToNamedGraph <http://localhost:' + str(port) + '/person' + str(c) + '/profile.ttl#MbPrivate>.\n'
    if auths == 3:
        b += ';\n\
    ppo:appliesToNamedGraph <http://localhost:' + str(port) + '/person' + str(c) + '/profile.ttl#MbPrivate>;\n\
    ppo:appliesToNamedGraph <http://localhost:' + str(port) + '/person' + str(c) + '/profile.ttl#MbOfficial>.\n'
    if auths > 3:
        print("Authorisations must be either 1, 2, or 3.", file=sys.stderr)
    return b

    #acl2 with access for person to this acl file
    #acl:mode acl:Write;\n\
    



In [14]:
#General functions

# Function for directory-creation
def create_dir(path):
    try:
        os.makedirs(path)
    except OSError:
        print ('Directory: ' + str(path) + ' already exists')
    else: 
        pass

    
# Function for document-creation    
def create_doc(filename, filecontent):
    # Creating & writing out documents
    f = open(filename, 'w')
    f.write(str(filecontent))
    f.close()
    

In [15]:
# Data set generating function
def generate(path_, task, subT, pods, posts, auths):
        
    # Path of Folder 
    path = './src/data/' + task + '/' + subT + '/'
    #create_dir(path)

    # Looping through one data set -> one loop per person
    for c in range(pods):
            
        # Creating directory for person
        path_person = path + 'person' + str(c)
        create_dir(path_person)

        # Creating profile document
        profile = ''
        profile += prefix() 
        profile += ':Card {\n'
        profile += body(c)
        profile += knows(pods)
        profile += account(c)
        profile += user(c, posts//2*2)

        # Creating half the posts for each microblog
        # -> Note: an odd number will be adjusted downwards

        post_count = 0
        profile += '}\n\n:MbPrivate {\n'
        for p in range(posts//2):
            profile += body_post(post_count)
            post_count += 1

        profile += '}\n\n:MbOfficial {\n'
        for p in range(posts//2):
            profile += body_post(post_count)
            post_count += 1

        profile += '}\n'

        # Creating path to profile document
        path_profile = path_person + '/profile.ttl'

        # Creating profile document
        create_doc(path_profile, profile)

        # Creating acl file, path and document    
        acl = ''
        acl += prefix_acl()
        acl += ':aclGraph {\n'
        acl += body_acl(c, auths, pods)
        acl += '}\n'

        path_acl = path_person + '/acl.ttl'
        create_doc(path_acl, acl)



In [16]:
#Calling function to greate data sets
for task in data:
    path = './data/'
    for subT in data[task]:
        path_subT = path + task + '/'+ subT + '/'
        generate(path_subT, task, subT, data[task][subT]['pods'], data[task][subT]['posts'], data[task][subT]['auths'])
        